In [1]:
import rasterio
import joblib
import pandas as pd
import json
from pathlib import Path
import numpy as np
import os


In [ ]:
# model specific info
from rural_beauty.config import data_dir, models_dir 
import importlib

module_name = "rural_beauty.config"
para_outcome = 'scenic'
para_type = 'linear'
sugar    =  '041124'  # random identifier to have different models with same other paras
country = 'UK'


model_basename = f"{country}_{para_outcome}_{para_type}_{sugar}"
model_folder = models_dir / model_basename


if not os.path.exists(models_dir / model_basename):
    os.mkdir(models_dir / model_basename)

model_path = model_folder / 'model.pkl'
scaler_X_path = model_folder / 'scaling_X.pkl'
scaler_Y_path = model_folder / 'scaling_Y.pkl'
significant_coefs_path = model_folder / "significant_coefs.csv"

output_raster_path = model_folder / 'prediction.tif'

# data specific info
feature_paths = models_dir / '_rnd_points' / country / 'feature_paths.json'





In [9]:
# load the model, the scaling and the significant coefficients. 
model = joblib.load(model_path)

# scaler_Y = joblib.load(scaler_Y_path)
significant_coefs = pd.read_csv(significant_coefs_path)


# Load the features paths dict
with open(feature_paths, 'r') as input_file:
    feature_filepaths = json.load(input_file)
    feature_filepaths = {k: Path(v) for k, v in feature_filepaths.items()} 

# these are only the features with non0 parameters after training. 
significant_filepaths = {feature : feature_filepaths[feature] for feature in significant_coefs.Feature}
coefficients = {k:v for k,v in zip(significant_coefs.Feature, significant_coefs.Coefficient.tolist())}

intercept = model.intercept_


In [4]:
# New base directory for the adjusted rasters
new_base_dir =  data_dir / 'forprediction'

new_base_dir.mkdir(parents=True, exist_ok=True)

# Create new list of file paths for the adjusted rasters
adjusted_raster_paths = {var:new_base_dir / model_basename / f"{raster_path.name}" for var, raster_path in significant_filepaths.items()}

os.makedirs(new_base_dir / model_basename, exist_ok=True)

adjusted_raster_paths
# testvar = 'dem_1'
# print(significant_filepaths[testvar])
# print(adjusted_raster_paths[testvar])
# print(coefficients         [testvar])


{'dem_3': PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/forprediction/unique_linear_041124/DEM_EU_range_scaled_zone3.tif'),
 'dem_1': PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/forprediction/unique_linear_041124/DEM_EU_range_scaled.tif'),
 'stoer_1_2': PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/forprediction/unique_linear_041124/code_stoer_zone1_2.tif'),
 'heide_1': PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/forprediction/unique_linear_041124/code_heide.tif'),
 'natgru_1_2': PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/forprediction/unique_linear_041124/code_natgru_zone1_2.tif'),
 'stra_1': PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/forprediction/unique_linear_041124/len_scaled_streets_EU_4647.tif'),
 'leit_1': PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/forprediction/unique_linear_041124/len_scaled_powerlines_EU_4647.tif'),
 'acker_1_2': PosixPath('/h/u145/hofer/MyDocuments/Granular/beau

In [5]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import joblib

def subset_and_align_rasters(raster_paths:dict, output_paths:dict, scaler_path:Path) -> None: 
    """
    Subset all rasters to the largest area contained within all inputs,
    align them to the same resolution and extent, and normalize the data using a saved scaler.

    Args:
    - raster_paths (list of str): List of file paths to the input rasters.
    - output_paths (list of str): List of output file paths to save the aligned and normalized rasters.
    - scaler_path (str): Path to the .pkl file containing the fitted StandardScaler.

    Returns:
    - None
    """

    # Open all rasters and get their bounds
    bounds_list = []
    transforms = []
    crs = None

    for path in raster_paths.values():
        with rasterio.open(path) as src:
            bounds_list.append(src.bounds)
            transforms.append(src.transform)
            if crs is None:
                crs = src.crs  # Assuming all rasters are in the same CRS, else reproject would be needed.

    # Calculate the largest common intersection (extent)
    intersection_bounds = (
        max([b.left for b in bounds_list]),   # Left bound
        max([b.bottom for b in bounds_list]), # Bottom bound
        min([b.right for b in bounds_list]),  # Right bound
        min([b.top for b in bounds_list])     # Top bound
    )

    ref_resolution = 1000
    pixel_width = ref_resolution
    pixel_height = ref_resolution
    width = int((intersection_bounds[2] - intersection_bounds[0]) / pixel_width)
    height = int((intersection_bounds[3] - intersection_bounds[1]) / pixel_height)

    # Align rasters to this common extent
    for (varname, raster_path), output_path in zip(raster_paths.items(), output_paths.values()):

        # make output dir if it doesn't exist already. 
        os.makedirs(output_path.parent, exist_ok=True)

        with rasterio.open(raster_path) as src:
            # Calculate the transform for the common extent
            transform, _, _ = calculate_default_transform(
                src.crs, src.crs, width=width, height=height, 
                left=intersection_bounds[0], bottom=intersection_bounds[1], 
                right=intersection_bounds[2], top=intersection_bounds[3])

            # Create a new dataset with the common extent
            kwargs = src.meta.copy()
            kwargs.update({
                'height': height,
                'width': width,
                'transform': transform
            })
            print(f"Aligning and normalizing {raster_path}. Dimensions: {width}, {height}")

            # Read, reproject, and normalize the data, then save
            with rasterio.open(output_path, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):  # Reproject and normalize all bands
                    # Allocate an array to hold the reprojected data
                    reprojected_data = np.empty((height, width), dtype=src.dtypes[i - 1])
                    
                    reproject(
                        source=rasterio.band(src, i),
                        destination=reprojected_data,
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.nearest  # You can change this to another method if needed
                    )

                    # Write the normalized data to the output file
                    dst.write(reprojected_data, i)






In [6]:
subset_and_align_rasters(significant_filepaths, adjusted_raster_paths, scaler_X_path)

Aligning and normalizing /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_scaled_zone3.tif. Dimensions: 6564, 4058
Aligning and normalizing /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/DEM_EU_range_scaled.tif. Dimensions: 6564, 4058
Aligning and normalizing /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_stoer_zone1_2.tif. Dimensions: 6564, 4058
Aligning and normalizing /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_heide.tif. Dimensions: 6564, 4058
Aligning and normalizing /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_natgru_zone1_2.tif. Dimensions: 6564, 4058
Aligning and normalizing /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/osm/len_scaled_streets_EU_4647.tif. Dimensions: 6564, 4058
Aligning and normalizing /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/osm/len_scaled_powerlines_EU_464

In [7]:
import rasterio
import numpy as np

def linear_combination_rasters(raster_paths:dict, coefficients:dict, constant:float, output_path: Path) -> None: 
    """
    Create a new raster from a linear combination of input rasters.
    
    Args:
    - raster_paths (list of str): List of file paths to the input rasters.
    - coefficients (list of float): Coefficients for the linear combination, same length as raster_paths.
    - output_path (str): File path to save the output raster.
    """

    first_layer_name = next(iter(adjusted_raster_paths))

    # Loop through the remaining rasters and apply the linear combination
    for varname, raster_path in raster_paths.items():
        
        with rasterio.open(raster_path) as src:
            feature_data = src.read(1, masked=True)
            maxvalue = feature_data.max()
            print(f"Maximum of {varname}: {maxvalue}")

            if varname == first_layer_name:
                meta = src.meta
                data = feature_data * coefficients[varname] # Read the first raster and multiply by its coefficient
            else:          
                data += feature_data * coefficients[varname]  # Add the weighted raster data

    data += constant
    # Update metadata if needed (e.g., datatype)
    meta.update(dtype=rasterio.float64)

    # Save the result as a new raster
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(data.astype(rasterio.float64), 1)




In [16]:
# now force it into the classes. 

def bin_raster(input_path: Path, output_path: Path, bins: list, nodata_value: int = -99) -> None:
    """
    Bin the values of a raster layer into classes, change data type to int, 
    and set the nodata value to -99.

    Args:
    - input_path (Path): Path to the input raster.
    - output_path (Path): Path to save the binned output raster.
    - bins (list): List of bin edges for classifying the raster values.
    - nodata_value (int): The new nodata value to assign in the output raster.
    """

    with rasterio.open(input_path) as src:
        # Read the raster data as float64 to handle potential NA and large values
        data = src.read(1).astype(np.float64)
        meta = src.meta

        # Create a mask for the existing nodata values in the input raster
        original_nodata = src.nodata
        nodata_mask = data == original_nodata

        # Bin the raster values
        binned_data = np.digitize(data, bins)

        # Apply the nodata mask and set the nodata value to -99
        binned_data = np.where(nodata_mask, nodata_value, binned_data).astype(np.int32)


        # Update metadata
        meta.update(dtype=rasterio.int32, nodata=nodata_value)

    # Write the binned raster to a new file
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(binned_data, 1)


In [11]:
constant = 0.5
linear_combination_rasters(adjusted_raster_paths, coefficients, constant, output_raster_path)

Maximum of dem_3: 0.45188230048380457
Maximum of dem_1: 0.9999999694371816
Maximum of stoer_1_2: 1.0000000000000002
Maximum of heide_1: 1.0
Maximum of natgru_1_2: 1.0000000000000002
Maximum of stra_1: 1.0
Maximum of leit_1: 1.0
Maximum of acker_1_2: 1.0000000000000002
Maximum of sgall_1: 1.0
Maximum of seemee_1: 1.0
Maximum of wein_1: 1.0


In [17]:
bins = [x/9 for x in range(1,9)] + [99]

binned_output_path = output_raster_path.with_name(f"binned_{output_raster_path.name}")
bin_raster(output_raster_path, binned_output_path, bins)

In [13]:
!gdalinfo -hist /h/u145/hofer/MyDocuments/Granular/beauty/data/models/unique_linear_041124/prediction.tif

Driver: GTiff/GeoTIFF
Files: /h/u145/hofer/MyDocuments/Granular/beauty/data/models/unique_linear_041124/prediction.tif
Size is 6564, 4058
Coordinate System is:
PROJCRS["ETRS89 / UTM zone 32N (zE-N)",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference F